In [504]:
import os
import re
import csv
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

In [505]:
data = pd.read_csv("data.csv",  encoding='utf8', low_memory=False )

In [506]:
data.shape

(4410, 12)

In [507]:
data.tail(2)

,_id,collective,date,file_id,judicature,participants,report_receipt_at,sentence_id,text,topic,url,vocabulary
4408,corteconstitucional_A. 175A/19,"{""entity"": {""$oid"": ""5fa5017c5821d011a2a92372""}}",2019-04-03T00:00:00.000Z,T-7071794,"[{""name"": ""ALBERTO ROJAS RÍOS"", ""rol"": ""PONENT...","[{""name"": ""DARIO GOMEZ SUAREZ"", ""role"": ""PLAIN...",2019-07-08,A. 175A/19,Auto 175A/19\n \n \nReferencia: Expediente T-7...,El accionante le solicitó a la Corporación dec...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 7}, {""text"": ""referen..."
4409,corteconstitucional_A. 121/19,"{""entity"": {""$oid"": ""5fa501875821d011a2a93763""}}",2019-03-13T00:00:00.000Z,T-6843600,"[{""name"": ""GLORIA STELLA ORTIZ DELGADO"", ""rol""...","[{""name"": ""COMISION INTERECLESIAL DE JUSTICIA ...",2019-03-21,A. 121/19,Auto\r\n121/19\n \n \nReferencia: expediente T...,Con el presente auto se suspenden los términos...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 17}, {""text"": ""refere..."


In [508]:
data = data.drop(["judicature", "participants", "collective", "_id", "topic", "url", "vocabulary", "date"], axis=1)

In [509]:
data.head()

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...
1,T-3183107,2012-05-18,T-016/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-016/...
2,T-3187308,2013-05-07,T-014/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-014/...
3,T-3182540,2012-05-18,T-015/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-015/12\...
4,T-2719663,2011-05-10,T-012/11,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-012/11\...


In [510]:
data.head(1)

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...


In [511]:
data["text"] = data["text"].str.lower()

In [512]:
len(data["text"])

4410

In [513]:
data

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,república de colombia\n\n\nsentencia t-017/12\...
1,T-3183107,2012-05-18,T-016/12,república de colombia\n\n\nsentencia\r\nt-016/...
2,T-3187308,2013-05-07,T-014/12,república de colombia\n\n\nsentencia\r\nt-014/...
3,T-3182540,2012-05-18,T-015/12,república de colombia\n\n\nsentencia t-015/12\...
4,T-2719663,2011-05-10,T-012/11,república de colombia\n\n\nsentencia t-012/11\...
...,...,...,...,...
4405,T-6403234,2019-04-08,A. 177/19,auto 177/19\n \nreferencia:\r\nexpediente t-64...
4406,T-6945381,2019-05-27,A. 177A/19,auto\r\n177a/19\n \n \nreferencia:\r\nexpedien...
4407,T-7064392 Y T-7073283,2019-04-08,SU.139/19,sentencia\r\nsu139/19 \n \naccion de tutela co...
4408,T-7071794,2019-07-08,A. 175A/19,auto 175a/19\n \n \nreferencia: expediente t-7...


In [514]:
new_data = pd.DataFrame([], columns = ['file_id', 'antecedentes' , 'decision'])

In [515]:
new_data

,file_id,antecedentes,decision


In [516]:
def quitar_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [517]:
def get_segment(data):
    lista_df = []
    for index, row in data.iterrows():
        texto = re.split(
            r'[\n]+[(^i|^v|)]+\.\s{0,}[(\w)]{0,}?', 
            quitar_tildes(row["text"])
        )
        if len(texto) > 4:
            lista_df.append({"file_id": row["file_id"] , "antecedentes": texto[1], "decision": texto[4]})

    return lista_df

In [518]:
new_data = pd.DataFrame(get_segment(data), columns = ["file_id", 'antecedentes' , 'decision'])

In [519]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes\n \n1. hechos\n \n1. el 27 de jul...,decision\n \ncon\r\nfundamento en las consider...
1,T-2719663,antecedentes\n \n1. hechos\n \nel defensor ...,"decision\n \nen merito de lo expuesto, la\r\ns..."
2,T-3176539,antecedentes\n \nel ocho (8) de junio de dos m...,"decision\n \nen merito de lo expuesto, la sala..."
3,T-3180007,"antecedentes\n \nel senor juan antonio, a trav...",consideraciones de la corte constitucional\n \...
4,T-2764991 Y 2822307,antecedentes\n \n1. hechos\n \nlos hechos rela...,"decision \n \nen merito de lo expuesto, la sal..."
...,...,...,...
1857,T-7128811,antecedentes\n \n1. hechos\n \n\n1.1 la senora...,que no esten\r\nexpresamente incluidos en el p...
1858,T-6960341,antecedentes \n \n1. hechos y relato contenido...,pruebas decretadas en sede de revision \n \nel...
1859,T-7147012,antecedentes\n \nherley\r\nramirez alzate pres...,existencia\r\nde un peligro especifico e indiv...
1860,T-7103649,antecedentes\n \na. hechos y pretensiones\n \n...,"decision\n \nen merito de lo expuesto, la sala..."


In [520]:
new_data.shape

(1862, 3)

In [521]:
new_data.to_csv('splid_data.csv', sep=",",quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')

In [522]:
new_data.to_csv(r'splid_data.txt', header=None, index=None, sep=' ', mode='a')

In [523]:
def quitar_espacios(text):
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    text = re.sub('\xa0', ' ', text)

    return text

new_data["decision"] = new_data.decision.apply(lambda x: quitar_espacios(x))
new_data["antecedentes"] = new_data.antecedentes.apply(lambda x: quitar_espacios(x))

In [524]:
#def quita_signos(text):
#    puntuacion = string.punctuation
#    for item in puntuacion:
#        text = text.replace(item, "")
#    return text

#new_data["decision"] = pd.DataFrame(new_data.decision.apply(lambda x: quita_signos(x)))
#new_data["antecedentes"] = pd.DataFrame(new_data.antecedentes.apply(lambda x: quita_signos(x)))

In [525]:
new_data[1:2]["antecedentes"]

1    antecedentes   1.    hechos   el defensor del ...
Name: antecedentes, dtype: object

In [526]:
new_data["decision"][50][0:2000]

'decision   en merito de lo expuesto, la  sala novena de revision de la corte constitucional, administrando justicia en  nombre del pueblo y por mandato de la constitucion,     resuelve:   primero.- revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado. en su lugar, conceder  la tutela por violacion del derecho al debido proceso.   segundo.- dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civil-familia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  s.a. contra la empresa construcciones dihago ltda. representada por alejandro  habeych gonzalez y contra las personas ana maria barrios queve

In [527]:
new_data["decision"][50]

'decision   en merito de lo expuesto, la  sala novena de revision de la corte constitucional, administrando justicia en  nombre del pueblo y por mandato de la constitucion,     resuelve:   primero.- revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado. en su lugar, conceder  la tutela por violacion del derecho al debido proceso.   segundo.- dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civil-familia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  s.a. contra la empresa construcciones dihago ltda. representada por alejandro  habeych gonzalez y contra las personas ana maria barrios queve

In [528]:
#from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
#import sparknlp
#import pyspark.sql.functions as F

#spark = sparknlp.start()
#spark

In [529]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [530]:
import sparknlp
spark = sparknlp.start()
from sparknlp.pretrained import PretrainedPipeline
spark


In [542]:
explain_document_pipeline = PretrainedPipeline("explain_document_md", lang = 'es')

explain_document_md download started this may take some time.
Approx size to download 453.9 MB
[OK!]


In [46]:
annotations = explain_document_pipeline.annotate(new_data["decision"][50])
list(annotations.keys())

['entities',
 'lemma',
 'document',
 'pos',
 'token',
 'ner',
 'embeddings',
 'sentence']

In [49]:
print('articulo' == annotations['token'])

False


In [50]:
def count_word(word):
    count=0
    for item in annotations["token"]:
        if word == item or f"{word}s" == item:
            count = count + 1
    return count

In [51]:
print("=====================")
print("Articulos:....... " + str(count_word('articulo')))
print("=====================")
print("Decretos:....... " + str(count_word('decreto')))
print("=====================")
print("Sentencias:....... " + str(count_word('sentencia')))
print("=====================")
print("Folios:....... " + str(count_word('folio')))
print("=====================")

Articulos:....... 12
Decretos:....... 5
Sentencias:....... 64
Folios:....... 6


In [55]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
  .pretrained("sentence_detector_dl", "en") \
  .setInputCols(["document"]) \
  .setOutputCol("sentences")

sd_pipeline = PipelineModel(stages=[documenter, sentencerDL])
sd_model = LightPipeline(sd_pipeline)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
def extraer_articulos(sentences, topic):
    lista_df = []
    for item in sentences:
        regx = f"({topic}|{topic}s)[ ]([\s\S]*?)\."
        expretion = re.compile(r"{}".format(regx))
        result = expretion.findall(item)
        for item_normalize in list(map(" ".join, result)):
            lista_df.append(item_normalize)
    return lista_df

In [ ]:

def get_list_sentenses():
    list_sentenses = []
    for sentense in sd_model.fullAnnotate(text)[0]["sentences"]:
        list_sentenses.append(sentense.result)
    return list_sentenses

In [533]:
text = new_data["decision"][50]

In [534]:
get_list_sentenses()[81]

'las entidades financieras quedan facultadas para  redimir en forma anticipada los titulos valores denominados en upac. igualmente,  a eleccion del deudor, se podran denominar las cuentas de ahorro y demas  pasivos, en uvr o en pesos.'

In [535]:
get_list_sentenses()[27:28]

['[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de tutela para reclamar ante los jueces, en todo momento y  lugar, mediante un procedimiento preferente y sumario, por si misma o por quien  actue a su nombre, la proteccion inmediata de sus derechos constitucionales  fundamentales, cuando quiera que estos resulten vulnerados o amenazados por la accion  o la omision de cualquier autoridad publica ().']

# Seccion DECISION [sentencias, articulos, folios, decretos]

In [536]:
decision_new_data_folio = pd.DataFrame(columns = ["folios"])
decision_new_data_articulo = pd.DataFrame(columns = ['articulos'])
decision_new_data_decreto = pd.DataFrame(columns = ['decretos'])
decision_new_data_sentencia = pd.DataFrame(columns = ['sentencias'])

In [537]:
decision_new_data_folio['folios'] = extraer_articulos(get_list_sentenses()[27:128], "folio")

In [538]:
decision_new_data_decreto['decretos'] = extraer_articulos(get_list_sentenses()[27:128], "decreto")

In [539]:
decision_new_data_articulo['articulos'] = extraer_articulos(get_list_sentenses()[27:128], "articulo")

In [540]:
decision_new_data_sentencia['sentencias'] = extraer_articulos(get_list_sentenses()[27:128], "sentencia")

In [541]:
decision_new_data_sentencia.head()

,sentencias
0,sentencia t-1223 de 2001 m
1,"sentencias c-543 de 1992, t-079 de 1993, t-23..."
2,sentencia c-590 de 2005 m
3,sentencia t-658 de 1998 m
4,sentencia c-590 de 2005 m


In [500]:
decision_new_data_articulo.head()

,articulos
0,articulo 86 de la c
1,articulo 319 del codigo de procedimiento civil
2,articulo 16 de la ley 31 de 1992
3,"articulos 18, 19, 20, 21, 22, 23, 134, 135, 1..."
4,articulo 121 del decreto ley 663 de 1993 (est...


In [502]:
decision_new_data_decreto.head()

,decretos
0,decreto ley 663 de 1993 (estatuto organico de...
1,decreto ley 663 de 1993 (estatuto organico de...
2,"decreto 856 de mayo 19 de 1999, (diario ofici..."
3,"decreto 2599 de 1998, especificando la denomi..."


In [503]:
decision_new_data_folio.head()

,folios


# Propuesta de generar NER para documentos Legales en Colombia

In [162]:
tokens=["articulo", "16", "de", "la", "ley", "31", "de", "1992"]
pos_labels=['ART', 'NUM', 'NN', 'NN', 'NN', 'NUM', 'NN', 'YEAR', '.']
entity_labels=['ARTICULO','NUM-ARTICULO','STOP','LEY','NUM-LEY','STOP','LEY-ANNO','O','O']

In [163]:
labels_lines=''
for token,pos,label in zip(tokens,pos_labels,entity_labels):
    labels_lines+="{} {} {} {}\n".format(token, pos, pos, label)
print(labels_lines)

articulo ART ART ARTICULO
16 NUM NUM NUM-ARTICULO
de NN NN STOP
la NN NN LEY
ley NN NN NUM-LEY
31 NUM NUM STOP
de NN NN LEY-ANNO
1992 YEAR YEAR O



In [223]:
addrtext = '[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de Articulo 12,23, 23,23. tutela para reclamar ante los jueces,'    
newtext = addrtext  
ippat = re.compile(r'(articulo|articulos)[ ]([\s\S]*?)\.', flags=re.I | re.X)  
mtch = ippat.search(newtext) 
print(ippat.findall(newtext))
print(mtch)



[('articulo', '86 de la c'), ('Articulo', '12,23, 23,23')]
<re.Match object; span=(8, 28), match='articulo 86 de la c.'>


In [224]:
import re
line = "[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de Articulo 12,23, 23,23. tutela para reclamar ante los jueces,"
matches = re.findall('(articulo|articulos)[ ]([\s\S]*?)\.', line)
print(matches)

[('articulo', '86 de la c')]


In [345]:
import re
s = "[15] el articulo 86 de la c. p. reza lo siguiente: toda persona tendra accion de articulo 12,23, 23,23. tutela para reclamar ante los jueces,."
r = re.compile(r'(articulo|articulos)[ ]([\s\S]*?)\.')
print(r.findall(s))

[('articulo', '86 de la c'), ('articulo', '12,23, 23,23')]
